Chris Holden (ceholden@gmail.com) - https://github.com/ceholden

Chapter 2: Your first remote sensing vegetation index
=====================================================

## 介绍
既然我们可以将数据读入计算机，那么让我们来计算一些植被指数。

[归一化植被指数（NDVI）](http://en.wikipedia.org/wiki/NDVI) 是如此普遍，甚至有一个维基百科条目。如果您是来学习如何使用 GDAL 和 Python 进行遥感图像处理的，我猜您对这一部分不需要任何介绍。如果您需要复习，请访问 [NDVI 的维基百科链接](http://en.wikipedia.org/wiki/NDVI)。


(1) What is NDVI (Normalized Difference Vegetation Index)?. https://gisgeography.com/ndvi-normalized-difference-vegetation-index/.
(2) Normalized difference vegetation index - Wikipedia. https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index.
(3) Landsat Normalized Difference Vegetation Index. https://www.usgs.gov/landsat-missions/landsat-normalized-difference-vegetation-index.

这一章非常直接。我们已经看到了如何将图像读入 NumPy 数组中 - 这一章将通过展示如何在 NumPy 对象上进行简单计算来扩展这些工具。

让我们回顾一下之前的代码，打开图像并读取数据：
```python
import gdal
import numpy as np

# 打开图像文件
image_path = "path/to/your/image.tif"
dataset = gdal.Open(image_path, gdal.GA_ReadOnly)

# 获取蓝色波段
blue = dataset.GetRasterBand(1)

# 获取波段数据
blue_data = blue.ReadAsArray()

# 打印波段数据的形状
print(f"波段数据形状：{blue_data.shape}")

# 打印波段数据的前几行
print("波段数据的前几行：")
print(blue_data[:5, :])
```
这将读取蓝色波段的数据并存储在名为 `blue_data` 的 NumPy 数组中。您可以根据需要进一步处理和分析这些数据。

In [1]:
# 导入 Python 3 的 print 函数
from __future__ import print_function

# 从 "osgeo" 模块中导入 "gdal" 和 "gdal_array" 子模块
from osgeo import gdal
from osgeo import gdal_array

# 导入 NumPy 模块
import numpy as np

# 打开一个 GDAL 数据集
dataset = gdal.Open('../../example/LE70220491999322EDC01_stack.gtif', gdal.GA_ReadOnly)

# 使用第一个波段的数据类型来分配我们的数组
image_datatype = dataset.GetRasterBand(1).DataType

image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                 dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))

# 循环遍历数据集中的所有波段
for b in range(dataset.RasterCount):
    # 注意，GDAL 的索引是从 1 开始的，而 Python 的索引是从 0 开始的 -- 因此我们在 GDAL 调用中要加 1
    band = dataset.GetRasterBand(b + 1)
    
    # 将波段的数据读取到我们数组的第三维中
    image[:, :, b] = band.ReadAsArray()
    

print('红色波段均值: {r}'.format(r=image[:, :, 2].mean()))
print('近红外波段均值: {nir}'.format(nir=image[:, :, 3].mean()))


红色波段均值: 589.379808
近红外波段均值: 3442.297712


即使是对整个图像进行简单的均值统计，我们也可以看到红色波段和近红外（NIR）波段之间的对比。

## 归一化植被指数（NDVI）

要计算 NDVI，我们可以在 Python 中使用标准的算术运算符，因为 NumPy 中的这些操作是矢量化的。就像 MATLAB、R 和其他高级语言一样，**永远不要**循环遍历 NumPy 数组，除非您可以避免这样做。

In [3]:
b_red = 2
b_nir = 3

ndvi = (image[:, :, b_nir] - image[:, :, b_red]) / (image[:, :, b_red] + image[:, :, b_nir])

print(ndvi)
print(ndvi.max())

[[0.71390828 0.71079741 0.69352291 ... 0.79392185 0.81408451 0.79165379]
 [0.68064263 0.6787194  0.6643924  ... 0.81387182 0.79880597 0.77389811]
 [0.66904762 0.67268446 0.66332892 ... 0.78495923 0.78278801 0.81253291]
 ...
 [0.68301262 0.68593651 0.67145614 ... 0.81065089 0.78050922 0.76519266]
 [0.67341718 0.6622986  0.65331611 ... 0.80436681 0.77483099 0.75      ]
 [0.63973799 0.62396514 0.66731813 ... 0.7094648  0.70005244 0.74574523]]
0.9046013008913515


#### 注意：Python 2

在 Python 2 中，整数除以整数会产生一个整数，即使除法本应产生一个浮点数。Python 3 改变了这种行为，但如果我们在 Python 2 中运行 NDVI 计算，我们将得到所有 NDVI 值都等于 0，因为我们的输入图像是整数数据类型（int16）。有关更多信息，请参阅 [NumPy 中除法的文档](http://docs.scipy.org/doc/numpy/reference/generated/numpy.divide.html)。

虽然在 Python 3 中我们不一定需要更改任何内容，但为了清晰起见，明确指定涉及计算的数据类型通常是有用的。此外，我们通常希望使用 Python 3 编写的代码也能在 Python 2 中运行。

为了确保我们使用浮点数执行计算，我们可以将计算的分子或分母转换为浮点数：
```python
# Calculate NDVI
red_band = image[:, :, 2]
nir_band = image[:, :, 3]

# Ensure division is performed using floating point numbers
ndvi = (nir_band.astype(float) - red_band.astype(float)) / (nir_band + red_band)

# Print NDVI statistics
print('NDVI mean: {ndvi_mean}'.format(ndvi_mean=ndvi.mean()))
print('NDVI min: {ndvi_min}'.format(ndvi_min=ndvi.min()))
print('NDVI max: {ndvi_max}'.format(ndvi_max=ndvi.max()))
```
这将计算 NDVI 并打印统计信息。如果您有其他问题或需要更多帮助，请随时告知！

In [4]:
ndvi = (image[:, :, b_nir] - image[:, :, b_red]) / \
        (image[:, :, b_nir] + image[:, :, b_red]).astype(np.float64)

print('NDVI matrix: ')
print(ndvi)

print('\nMax NDVI: {m}'.format(m=ndvi.max()))
print('Mean NDVI: {m}'.format(m=ndvi.mean()))
print('Median NDVI: {m}'.format(m=np.median(ndvi)))
print('Min NDVI: {m}'.format(m=ndvi.min()))

NDVI matrix: 
[[0.71390828 0.71079741 0.69352291 ... 0.79392185 0.81408451 0.79165379]
 [0.68064263 0.6787194  0.6643924  ... 0.81387182 0.79880597 0.77389811]
 [0.66904762 0.67268446 0.66332892 ... 0.78495923 0.78278801 0.81253291]
 ...
 [0.68301262 0.68593651 0.67145614 ... 0.81065089 0.78050922 0.76519266]
 [0.67341718 0.6622986  0.65331611 ... 0.80436681 0.77483099 0.75      ]
 [0.63973799 0.62396514 0.66731813 ... 0.7094648  0.70005244 0.74574523]]

Max NDVI: 0.9046013008913515
Mean NDVI: 0.7088133953809207
Median NDVI: 0.7319195214790647
Min NDVI: 0.09470304975922954


这看起来是正确的。

说到“看起来正确”，下一章（[网页链接](chapter_3_visualization.html)或[Notebook链接](chapter_3_visualization.ipynb)）将演示如何使用实际的图表来可视化您的结果！